# Soil moisture mapping module

In [1]:
#set user libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Image
from ipywidgets import HBox, Label
import subprocess
# import ipyleaflet

import pysmm,os
from pysmm.derive_SM import get_map
import sys
import ee
import itertools
from IPython.core.display import HTML 


ee.Initialize()
path = os.path.dirname(pysmm.__file__)

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


# Introduction

In [2]:
user = path.split("/")[2]
print "Welcome " +  user + "!" + " Run this application to create soil moisture maps from Sentinel 1 imagery."

Welcome finegold! Run this application to create soil moisture maps from Sentinel 1 imagery.


# Set user variables

## Define the area of interest
### Provide the link to a Google Earth Engine asset and define the column name and value to select

In [3]:
aoi =  widgets.Text(
    value='users/yelenafinegold/107_phu_merged',
    placeholder='Enter your asset ID here',
#    description='Google Earth Engine asset ID:',
    disabled=False
#    , continuous_update=True
)
ui1 = HBox([Label('Google Earth Engine asset ID: '),aoi])
display(ui1)

SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdHb29nbGUgRWFydGggRW5naW5lIGFzc2V0IElEOiAnKSwgVGV4dCh2YWx1ZT11J3VzZXJzL3llbGVuYWZpbmVnb2xkLzEwN19waHVfbWVyZ2XigKY=


In [4]:
studyArea = ee.FeatureCollection(aoi.value)
col = ee.Feature(studyArea.first()).propertyNames().getInfo()
column_name =  widgets.Select(
    options=col,
#     value='id',
    placeholder='column name from the GEE asset',
#    description='Column name:',
    disabled=False
#     , continuous_update=True
)
ui2 = HBox([Label('Column name: '),column_name])
display(ui2)

SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdDb2x1bW4gbmFtZTogJyksIFNlbGVjdChvcHRpb25zPSh1J3N5c3RlbTppbmRleCcsIHUnaWQnKSwgdmFsdWU9dSdzeXN0ZW06aW5kZXgnKSnigKY=


In [8]:
colval = studyArea.distinct(column_name.value).aggregate_array(column_name.value).getInfo()
# print(colval)
column_value = widgets.Select(
    options=colval,
#     value=str(colval),
    placeholder='column value from the GEE asset',
#    description='Column value:',
    disabled=False
#     , continuous_update=True
)
ui3 = HBox([Label('Column value: '),column_value])
display(ui3)

SEJveChjaGlsZHJlbj0oTGFiZWwodmFsdWU9dSdDb2x1bW4gdmFsdWU6ICcpLCBTZWxlY3Qob3B0aW9ucz0oMTExNCwgMTExNSwgMTExNiwgMTExNywgMTExOCwgMTExOSwgMTEyMCwgMTExMSzigKY=


In [6]:
# Modify the Year, Month, and Day parameters as desired
year = widgets.SelectMultiple(
    options=[2014,2015,2016,2017,2018,2019],
    value=[2015,2016,2017,2018,2019],
    #rows=10,
    description='Year',
    disabled=False
)

display(year)

month = widgets.SelectMultiple(
    options=[('January',1),('February',2),('March',3),('April',4),('May',5),('June',6),('July',7),('August',8),('September',9),('October',10),('November',11),('December',12)],
    value=[4,10],
    description='Month',
    disabled=False
)

display(month)

day = widgets.SelectMultiple(
    options=range(1,32),
    value=[1],
    description='Day',
    disabled=False
)

display(day)



U2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdZZWFyJywgaW5kZXg9KDEsIDIsIDMsIDQsIDUpLCBvcHRpb25zPSgyMDE0LCAyMDE1LCAyMDE2LCAyMDE3LCAyMDE4LCAyMDE5KSwgdmFsdWXigKY=


U2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdNb250aCcsIGluZGV4PSgzLCA5KSwgb3B0aW9ucz0oKCdKYW51YXJ5JywgMSksICgnRmVicnVhcnknLCAyKSwgKCdNYXJjaCcsIDMpLCAoJ0HigKY=


U2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdEYXknLCBpbmRleD0oMCwpLCBvcHRpb25zPSgxLCAyLCAzLCA0LCA1LCA2LCA3LCA4LCA5LCAxMCwgMTEsIDEyLCAxMywgMTQsIDE1LCAxNizigKY=


# Run the PYSMM code for a specified AOI - will iterate over all combinations of Year / Month / Day sequentially.

In [9]:
studyArea0 = ee.FeatureCollection(aoi.value)
attribute = column_name.value
attribute_value = float(column_value.value)
studyArea1 = studyArea0.filterMetadata(attribute,'equals',attribute_value).geometry().bounds().coordinates()
coords = studyArea1.get(0).getInfo()
ll = coords[0]
ur = coords[2]
minlon = ll[0]
minlat = ll[1]
maxlon = ur[0]
maxlat = ur[1]
button = widgets.Button(description='Run SMM')
# button, output, function and linkage
outt = widgets.Output()
print('Asset: ' + aoi.value)
print('Column: ' + attribute)
print('Column value: ' + str(column_value.value))
print('Years: ' + str(year.value))
print('Months: ' + str(month.value))
print('Days: ' + str(day.value))
def format_date(x):
    if type(x) is int:
        l = [x]
    elif len(x) < 2:
        l = [x[0]]
    else:
        l = list(x)
    return [str(i) for i in l]  

year0= format_date(year.value)
year1 = ', '.join(year0)
month0= format_date(month.value)
month1 = ', '.join(month0)
day0= format_date(day.value)
day1 = ', '.join(day0) 
#         attribute_value0 = format_date(column_value.value)
#         attribute_value1 = ', '.join(attribute_value0)
studyArea0 = aoi.value
#         attribute = column_name.value
attribute_values1 = str(attribute_value).split('.')[0]
    
def on_button_clicked(b):
    with outt:
        clear_output()
        print('Processing and downloading soil moisture maps. This takes a long time')

        print(attribute_values1)
        print(studyArea0)
#         !mv ./nohup.out ./nohup_backup.out
        py = sys.executable
        !bash ./run_smm/run_pysmm.sh $py "$year1" "$month1" "$day1" $minlon $minlat $maxlon $maxlat $attribute_values1
        print('done!')
button.on_click(on_button_clicked)
# display
widgets.VBox([button,outt])


Asset: users/yelenafinegold/107_phu_merged
Column: id
Column value: 1120
Years: (2019,)
Months: (10,)
Days: (1, 15)


In [ ]:

# print('To run the script in command line copy the line of code below ' )
# print('cd  /home/'+user+'/smm')
# print('source env/bin/activate')
# print('nohup python /home/'+user+'/ws_idn_20190819/scripts/run_pysmm_ui.py '+ str(year1) +' ' + str(month1) +' ' + str(day1)+' '  + str(minlon)+' '  + str(minlat) +' ' + str(maxlon)+' '  + str(attribute_values1) +' &')

In [ ]:
# Results will be found in your Google Drive